# Simple demonstration of application of `Numba` to speed up python codes

In [1]:
import time
import numpy as np
from numba import jit

## A simple mock-up of the `cross-correlation` script

In [2]:
def cross_correlate(d, y, dt=0.01):
    ii = 0
    time_slides = []
    M = []
    while len(d[ii:]) >= len(y):
        time_slides.append(ii*dt)
        M.append(np.sum(d[ii: len(y) + ii] * y))
        ii += 1
    M = np.array(M)
    time_slides = np.array(time_slides)
    return (time_slides, M)

## `Numba` decorated version of the above function

In [3]:
@jit(nopython=True) # The only new thing requried
def cross_correlate_numba(d, y, dt=0.01):
    ii = 0
    time_slides = []
    M = []
    while len(d[ii:]) >= len(y):
        time_slides.append(ii*dt)
        M.append(np.sum(d[ii: len(y) + ii] * y))
        ii += 1
    M = np.array(M)
    time_slides = np.array(time_slides)
    return (time_slides, M)

### Make some fake data and template

In [4]:
d = np.random.random(500000)
y = np.random.random(100)

### Speed test the standard code

In [5]:
%%timeit
cross_correlate(d, y)

2.35 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Speed test `Numba` script

In [6]:
%%timeit
cross_correlate_numba(d, y)

131 ms ± 3.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Sanity check that the results are same

In [7]:
m_standard = cross_correlate(d, y)
m_numba = cross_correlate_numba(d, y)
print(m_standard[0] - m_numba[0])
print(m_standard[1] - m_numba[1])

[0. 0. 0. ... 0. 0. 0.]
[ 7.10542736e-15  0.00000000e+00 -7.10542736e-15 ...  0.00000000e+00
  7.10542736e-15 -7.10542736e-15]


In [8]:
np.max(np.abs(m_standard[1] - m_numba[1]))

3.552713678800501e-14